# 3. ) Proceso de fabricación de dulces: 
   
Inicio: 1 máquina provee chicles, 1 máquina que provee 1 unidad de polvo para cubrir el chicle
    
* Proceso 1: Maquina que recibe el chicle y el polvo, la cual cubre el chicle con el polvo
    
* Proceso 2: Maquina que recibe el bolichicle y lo envuelve
    
* Proceso 3: Maquina que toma 10 bolichicles envueltos para envasarlos de 10 en 1 bolsa
    
Se deberá simular el proceso de fabricación dando tiempos para cada proceso y sincronizando cada uno

In [1]:
import threading
import queue
import random

### Creando nuestras clase Mutex


In [2]:
class Mutex:
    
    def __init__(self):
        self.lock = threading.Lock()
    
    def bloquea(self):
        self.lock.acquire()
    
    def libera(self):
        self.lock.release()

### Creando nuestra clase Maquina1, que será la encargada de recibir el chicle y el polvo

In [3]:
class Maquina1(threading.Thread):
    
    def __init__(self,mutex,grupo):
        threading.Thread.__init__(self)
        self.mutex = mutex
        self.Grupo = grupo
    def espolvorea(self):
        
        global bolichicle
        
        print("Chicle espolvoreado")
        
        #Indicamos que ya creamos nuestro bolichicle
        bolichicle = True

    def run(self):
        global chicle
        global polvo
        
        while chicle & polvo :
            
            self.mutex.bloquea()
            
            print("\nMáquina 1 <active>")
            self.espolvorea()
            self.mutex.libera()
            polvo = False
            chicle = False

        
            

### Creando nuestra clase Maquina 2, que será la encargada de recibir el bolichicle y envolverlo

In [4]:
class Maquina2(threading.Thread):
    
    def __init__(self,mutex,Grupo):
        threading.Thread.__init__(self)
        self.mutex = mutex
        self.Grupo = Grupo
        
    def Envuelve(self):
        global bolichicle
        print("Envolviendo Bolichicle")
        bolichicle = False

    def Agrupa_bolichicle(self,grupo):
        
        grupo.put('Bolichicle')
        print("Juntando bolichicles, llevamos:",grupo.qsize())
    
    def run(self):
        global chicle
        global polvo
        global bolichicle
        
        while bolichicle:
            if self.Grupo.qsize()<=10:
                self.mutex.bloquea()
           # while bolichicle == True and self.Grupo.qsize() <10:
                print("\nMaquina 2 <active>")
                self.Envuelve()

                self.Agrupa_bolichicle(self.Grupo)
                        #Ya no existe un bolichicle, pues lo tenemos nosotros
                        #Debemos producir otro
                if self.Grupo.qsize() +1 == 11:
                    chicle = False
                    polvo = False
                else:
                    chicle = True
                    polvo = True
                self.mutex.libera()
            

### Creando nuestra Maquina 3, que toma 10 bolichicles envueltos para envasarlos de 10 en 1 bolsa

In [5]:
class Maquina3(threading.Thread):
    
    def __init__(self,mutex,Grupo):
        
        threading.Thread.__init__(self)
        self.mutex = mutex
        self.Grupo = Grupo
        
    def Envasa(self,Grupo):
        print("La bolsa contiene:")
        for i in range(10):
            print(Grupo.get('Bolichicle')," ",i+1)
        
        
    def run(self):
        global p
        
        while self.Grupo.qsize() ==10:
            self.mutex.bloquea()
            
            print("\nMaquina 3 <Active>")
            print("\nAgrupando de 10 en 10")
            self.Envasa(self.Grupo)
            
            #Termina nuestro ciclo
            p = False
            
            self.mutex.libera()

### Ejecutor (main)

In [6]:
mutex = Mutex()

Grupo = queue.Queue(10)
#Declaramos nuestros ingredientes
bolichicle = False
chicle = True
polvo = True
p = True

while p:
    M1 = Maquina1(mutex,Grupo)
    M2 = Maquina2(mutex,Grupo)
    M3 = Maquina3(mutex,Grupo)

    M1.start()
    M2.start()
    M3.start()
    M1.join()
    M2.join()
    M3.join()


Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 1

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 2

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 3

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 4

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 5

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 6

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 7

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envolviendo Bolichicle
Juntando bolichicles, llevamos: 8

Máquina 1 <active>
Chicle espolvoreado

Maquina 2 <active>
Envo